In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utils.functions import img2real
import json
import pickle
import numpy as np

In [2]:
X = pd.read_csv('data/X.csv')
X

,x,y,z,xe,ye,ze
0,0.00000,0.000000,0.00,0.000000,0.000000,0.0
1,0.00000,0.000000,0.00,0.000000,0.000000,0.0
2,0.00000,0.000000,0.00,0.000000,0.000000,0.0
3,0.15398,-0.112200,0.12,0.014152,0.047550,0.0
4,0.00000,0.000000,0.00,0.000000,0.000000,0.0
...,...,...,...,...,...,...
4047,0.16352,-0.081858,0.12,0.003757,-0.009588,0.0
4048,0.14995,-0.043175,0.12,0.011834,-0.029064,0.0
4049,0.16150,-0.064625,0.12,0.003944,-0.020419,0.0
4050,0.16352,-0.081858,0.12,0.003757,-0.009588,0.0


In [5]:
y = pd.read_csv('data/y.csv')
y

,t0,t1,t2,t3,t4,t5
0,-0.305360,1.5018,-1.3873,0.060087,0,0
1,-0.155500,1.4828,-1.3671,0.058877,0,0
2,-0.141360,1.4745,-1.3582,0.058254,0,0
3,-0.081871,1.4284,-1.3077,0.053806,0,0
4,-0.081871,1.4284,-1.3077,0.053806,0,0
...,...,...,...,...,...,...
1008,-0.182560,1.5801,-1.4675,0.061984,0,0
1009,-0.295830,1.4827,-1.3671,0.058875,0,0
1010,-0.355970,1.4248,-1.3037,0.053386,0,0
1011,-0.355970,1.4248,-1.3037,0.053386,0,0


In [2]:
df = pd.read_csv('data/train_data.csv')
df = df.dropna()
#df['error'] = np.sqrt((df['x']-df['xc'])**2 + (df['y']-df['yc'])**2)
df['xe'] = df['x']-df['xc']
df['ye'] = df['y']-df['yc']

min_max_scalers = {}

for column in df.columns:
    if column in  ['xc', 'yc', 'xe', 'ye']:
        min_max_scalers[column] = MinMaxScaler() #(df[column].min(), df[column].max())
        df[column] = min_max_scalers[column].fit_transform(df[[column]])
        with open('scalers/'+column+'.pkl', 'wb') as f:
            pickle.dump(min_max_scalers[column], f)

#with open('data/min_max_scalers.json', 'w') as f:
#    json.dump(min_max_scalers, f)

df.to_csv(f"data/train_data_clean.csv", index=False) 

df.head()

,x,y,z,t0,t1,t2,t3,xc_px,yc_px,xc,yc,diagonal,xe,ye
7,0.16,-0.19,0.12,30.100908,33.948155,75.041009,41.092854,145.0,-275.0,0.591045,0.987826,73.389373,0.411584,0.012639
8,0.17,-0.18,0.12,33.363423,34.952635,76.655631,41.702995,160.0,-264.0,0.635821,0.968696,72.801099,0.367291,0.031970
9,0.18,-0.17,0.12,36.636577,34.952635,76.655631,41.702995,178.0,-249.0,0.689552,0.942609,72.780492,0.313799,0.057993
10,0.19,-0.16,0.12,39.899092,33.948155,75.041009,41.092854,191.0,-235.0,0.728358,0.918261,72.835431,0.275639,0.082342
11,0.20,-0.15,0.12,43.130102,31.753588,71.469230,39.715641,208.0,-222.0,0.779104,0.895652,75.239617,0.225213,0.105019


In [36]:
pos_inicial = img2real((314,134),27) # pos_inicial e cameraHeight utilizado
print(pos_inicial)

#error_start = np.sqrt((0.035-pos_inicial[0])**2 + (0-pos_inicial[1])**2)
xe_start = 0.035-pos_inicial[0]
ye_start = 0-pos_inicial[1]
print(xe_start, ye_start)

pos_inicial = np.array([
    min_max_scalers['xc'].transform(np.array(pos_inicial[0]).reshape(1, -1))[0][0],
    min_max_scalers['yc'].transform(np.array(pos_inicial[1]).reshape(1, -1))[0][0],
    min_max_scalers['xe'].transform(np.array(xe_start).reshape(1, -1))[0][0],
    min_max_scalers['ye'].transform(np.array(ye_start).reshape(1, -1))[0][0]
])

print(pos_inicial)

(-5.652, 2.412)
5.687 -2.412
[-0.77910448  0.7426087   1.79778535  0.266171  ]


c:\Users\pedro\Documents\NOVOdeltadelta\pog\mlp-robot\.auto\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\pedro\Documents\NOVOdeltadelta\pog\mlp-robot\.auto\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\pedro\Documents\NOVOdeltadelta\pog\mlp-robot\.auto\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\pedro\Documents\NOVOdeltadelta\pog\mlp-robot\.auto\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [47]:
row_start = df.iloc[2]
row_start[['xc','yc','xe','ye']].values.shape

(4,)

In [3]:
from random import randint
import random
random.seed(42)

In [4]:
from IPython.display import clear_output

In [5]:
def perc_pos(tgt, stt, perc):
    return int((((tgt-stt)*perc)+stt))/100


input_data = torch.Tensor()
output_data = torch.Tensor()
count = 0
paths = []
NUM_PATHS = 1000

for _ in range(NUM_PATHS):
     #row = df.iloc[i]
     check = True
     while check:
          print('tentou')
          idx_s = randint(0, len(df)-1)
          idx_t = randint(0, len(df)-1)
          check = (idx_s == idx_t) or (set([idx_s,idx_t]) in paths) or (abs(df.iloc[idx_s]['x']-df.iloc[idx_t]['x'])<0.05) or (abs(df.iloc[idx_s]['y']-df.iloc[idx_t]['y'])<0.05)

     paths.append(set([idx_s,idx_t]))

     row_start = df.iloc[idx_s]
     x_start, y_start = row_start['x']*100, row_start['y']*100
     pos_hist = torch.Tensor(row_start[['xc', 'yc', 'xe', 'ye']].values.reshape(1,-1))

     row_fin = df.iloc[idx_t]
     x_fin, y_fin = row_fin['x']*100, row_fin['y']*100

     #pos_hist = torch.Tensor(pos_inicial)
     for p in [0.25, 0.5, 0.75]:
          check=True
          c = 0
          while(check):
               path_x = perc_pos(x_fin,x_start,p)
               path_y = perc_pos(y_fin,y_start,p)
               pos = df.loc[(df.x==path_x)&(df.y==path_y),['xc','yc','xe','ye']].values
               if(pos.size==0):
                    p += 0.05
                    if(c==20):
                         print(x_start, y_start)
                         print(x_fin, y_fin)
                         print(p)
                         print(path_x, path_y)
                         print(pos)
                    elif(c<20):
                         print(p)
               else:
                    check = False
               c+=1
          pos_hist = torch.cat((pos_hist, torch.Tensor(pos)))
     input_data = torch.cat((input_data, pos_hist.unsqueeze(0)))
     output_data = torch.cat((output_data, torch.Tensor([row_fin[['t0','t1','t2','t3']].values])))
     clear_output(wait=True)
     print(f'{count} path added')
     count +=1

input_data
     

999 path added


tensor([[[0.1731, 0.4087, 0.8256, 0.5929],
         [0.2537, 0.4817, 0.7462, 0.5190],
         [0.3254, 0.5478, 0.6760, 0.4517],
         [0.4358, 0.6313, 0.5659, 0.3677]],

        [[0.8358, 0.6765, 0.1687, 0.3232],
         [0.6090, 0.4957, 0.3949, 0.5037],
         [0.4627, 0.3757, 0.5400, 0.6247],
         [0.2776, 0.2243, 0.7233, 0.7768]],

        [[0.8955, 0.2870, 0.1107, 0.7119],
         [0.6567, 0.4174, 0.3475, 0.5827],
         [0.4687, 0.5078, 0.5339, 0.4920],
         [0.2985, 0.5774, 0.7019, 0.4214]],

        ...,

        [[0.2269, 0.7339, 0.7721, 0.2643],
         [0.3552, 0.5826, 0.6453, 0.4164],
         [0.4687, 0.4643, 0.5339, 0.5357],
         [0.5910, 0.3061, 0.4133, 0.6935]],

        [[0.7582, 0.2713, 0.2467, 0.7279],
         [0.6000, 0.3548, 0.4041, 0.6448],
         [0.5104, 0.4191, 0.4927, 0.5810],
         [0.4179, 0.4904, 0.5843, 0.5097]],

        [[0.5910, 0.3061, 0.4133, 0.6935],
         [0.4687, 0.4643, 0.5339, 0.5357],
         [0.3552, 0.5826, 0.64

In [6]:
output_data

tensor([[ 46.3099,  81.3969, 136.9517,  55.5547],
        [150.7100,  62.7777, 116.2649,  53.4872],
        [ 26.8699, 128.7312, 168.1344,  39.4032],
        ...,
        [116.8699,  31.7536,  71.4692,  39.7156],
        [ 74.8056, 122.3657, 165.6356,  43.2698],
        [  7.3540,  88.6237, 143.6680,  55.0444]])

In [7]:
BATCH_SIZE = 8

X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.2, random_state=42, shuffle=True)

X_test = X_test[4:,:,:]
y_test = y_test[4:,:] # p/ evitar data leakage

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=BATCH_SIZE)

# Separação em conj de validação
#X_train, y_train, X_val, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42, shuffle=True)

#train_loader

---
---
---

In [10]:
# Dados de exemplo
# Suponha batch_size = 1, seq_len = 4, input_size = 2 (x, y)
from itertools import product
from utils.rnn import ControlRNN, train_net
import torch
import torch.nn as nn

In [6]:
X_tensor = torch.Tensor([])
y_tensor = torch.Tensor([])

for group in range(len(y)):
    X_tensor = torch.cat([X_tensor, torch.Tensor(X.loc[list(range(4*group,4*(group+1))),['x','y','xe','ye']].values).reshape(1,4,4)])
    y_tensor = torch.cat([y_tensor, torch.Tensor(y.loc[group,['t0','t1','t2','t3']].values).reshape(1,4)])

In [7]:
X_tensor

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.1540, -0.1122,  0.0142,  0.0476]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.1540, -0.1122,  0.0142,  0.0476],
         [ 0.1667, -0.0707,  0.0088,  0.0306]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.1540, -0.1122,  0.0142,  0.0476],
         [ 0.1667, -0.0707,  0.0088,  0.0306],
         [ 0.1728, -0.0407,  0.0064,  0.0129]],

        ...,

        [[ 0.1096,  0.1003,  0.0302, -0.0957],
         [ 0.1428,  0.0063,  0.0117, -0.0530],
         [ 0.1499, -0.0432,  0.0118, -0.0291],
         [ 0.1615, -0.0646,  0.0039, -0.0204]],

        [[ 0.1428,  0.0063,  0.0117, -0.0530],
         [ 0.1499, -0.0432,  0.0118, -0.0291],
         [ 0.1615, -0.0646,  0.0039, -0.0204],
         [ 0.1635, -0.0819,  0.0038, -0.0096]],

        [[ 0.1499, -0.0432,  0.0118,

In [8]:
BATCH_SIZE = 8

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42, shuffle=True)

X_test = X_test[4:,:,:]
y_test = y_test[4:,:] # p/ evitar data leakage

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=BATCH_SIZE)

# Separação em conj de validação
#X_train, y_train, X_val, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42, shuffle=True)

#train_loader

In [11]:
# Inicializa a RNN
criterion = nn.MSELoss()

In [14]:
from tqdm.auto import tqdm
hidden_size = [300, 400]
num_layers = [1,2]
dropout_rate = [0,0.25,0.75]
learning_rate = [0.0001, 0.001, 0.01]
weight_decay = [0, 0.0005]
#optimizer = ['adam', 'sgd']
best_model = None
best_score = None

params_grid = product(hidden_size, num_layers, dropout_rate, learning_rate, weight_decay)

count = 1
for hs, nl, dr, lr, wd in tqdm(params_grid):
    if(not((dr>0) and (nl==1))):
        model = ControlRNN(input_size=4, hidden_size=hs, 
                        num_layers=nl, dropout_rate=dr)
        #if(o=='adam'):
        opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        #else:
        #    opt = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=wd)
        
        
        model, _, scores = train_net(model, train_loader, test_loader, 30, opt, verbose=0)
        score = np.mean(scores[1])
        print(f'[{count}] Val-MSE: {score:.3f}')
        if(best_score is None or best_score > score):
            print('[Melhor modelo atualizado]')
            print('hs:{}, nl:{}, dr:{}, lr:{}, wd:{}'.format(hs, nl, dr, lr, wd))
            best_model = model
            best_opt = opt
            best_score = score
        count += 1
                                

# Teste
predicted = best_model(input_data).detach()
print("Predição:", predicted)

[1] Val-MSE: 0.271
[Melhor modelo atualizado]
hs:300, nl:1, dr:0, lr:0.0001, wd:0
[2] Val-MSE: 0.279
[3] Val-MSE: 0.241
[Melhor modelo atualizado]
hs:300, nl:1, dr:0, lr:0.001, wd:0
[4] Val-MSE: 0.226
[Melhor modelo atualizado]
hs:300, nl:1, dr:0, lr:0.001, wd:0.0005
[5] Val-MSE: 0.267
[6] Val-MSE: 0.261
[7] Val-MSE: 0.267
[8] Val-MSE: 0.271
[9] Val-MSE: 0.273
[10] Val-MSE: 0.256
[11] Val-MSE: 0.267
[12] Val-MSE: 0.263
[13] Val-MSE: 0.266
[14] Val-MSE: 0.273
[15] Val-MSE: 0.268
[16] Val-MSE: 0.258
[17] Val-MSE: 0.266
[18] Val-MSE: 0.263
[19] Val-MSE: 0.264
[20] Val-MSE: 0.277
[21] Val-MSE: 0.269
[22] Val-MSE: 0.259
[23] Val-MSE: 0.267


KeyboardInterrupt: 

In [ ]:
hs:300, nl:1, dr:0, lr:0.001, wd:0.0005

In [21]:
full_loader = DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=BATCH_SIZE)
rnn_model = ControlRNN(input_size=4, hidden_size=300, num_layers=1, dropout_rate=0)
opt = torch.optim.Adam(rnn_model.parameters(), lr=0.001, weight_decay=0.0005)
rnn_model, _, scores = train_net(rnn_model, full_loader, full_loader, 60, opt, verbose=2)

Epoch [1/60], Train Loss: 0.6970, Eval Loss: 0.2655
best_model updated
Epoch [2/60], Train Loss: 0.3509, Eval Loss: 0.2651
best_model updated
Epoch [3/60], Train Loss: 0.3507, Eval Loss: 0.2652
Epoch [4/60], Train Loss: 0.3500, Eval Loss: 0.2649
best_model updated
Epoch [5/60], Train Loss: 0.3493, Eval Loss: 0.2645
best_model updated
Epoch [6/60], Train Loss: 0.3487, Eval Loss: 0.2642
best_model updated
Epoch [7/60], Train Loss: 0.3481, Eval Loss: 0.2639
best_model updated
Epoch [8/60], Train Loss: 0.3476, Eval Loss: 0.2636
best_model updated
Epoch [9/60], Train Loss: 0.3471, Eval Loss: 0.2633
best_model updated
Epoch [10/60], Train Loss: 0.3466, Eval Loss: 0.2630
best_model updated
Epoch [11/60], Train Loss: 0.3460, Eval Loss: 0.2626
best_model updated
Epoch [12/60], Train Loss: 0.3455, Eval Loss: 0.2624
best_model updated
Epoch [13/60], Train Loss: 0.3449, Eval Loss: 0.2620
best_model updated
Epoch [14/60], Train Loss: 0.3443, Eval Loss: 0.2617
best_model updated
Epoch [15/60], Train

In [17]:
predicted = rnn_model(X_test).detach()

score = ((predicted - y_test)**2).mean()

print(f'''MSE: {score:.4f}
RMSE: {score**0.5:.4f}''')

MSE: 0.0046
RMSE: 0.0677


In [22]:
predicted = rnn_model(X_test).detach()

score = ((predicted - y_test)**2).mean()

print(f'''MSE: {score:.4f}
RMSE: {score**0.5:.4f}''')

MSE: 0.0460
RMSE: 0.2145


In [29]:
predicted = best_model(X_test).detach()

score = ((predicted - y_test)**2).mean()

print(f'''MSE: {score:.4f}
RMSE: {score**0.5:.4f}''')

MSE: 40.5490
RMSE: 6.3678


In [31]:
best_model

ControlRNN(
  (rnn): RNN(3, 300, num_layers=2, batch_first=True)
  (fc): Linear(in_features=300, out_features=4, bias=True)
)

In [34]:
best_model.dropout_rate

0

Melhor modelo:

- Número de camadas recorrentes: 2
- Número de neurônios ocultos: 300
- Funcção de ativação: ReLU
- Taxa de dropout: 0
- Ottimizador: Adam
- Learning rate: 0.01
- Weight decay: 0.0005

In [23]:
#with open(, 'wb') as f:
best_model.eval()
torch.save(best_model.state_dict(), 'models/nn_model.pt')